- measure max feature exposure


- measure cum corr

- check ay metrics presented on the colab

Still unknown to do

- ensemble method 

- fit on different targets ?

- All the production automatization part

In [ ]:
The practical workflow:
Phase 1 (Feature selection - THIS WEEK):

Use fixed train/val split
Test feature counts quickly
Pick best configuration

Phase 2 (Final validation - NEXT WEEK):

Take your best feature set
Run the rolling simulation above
This gives you realistic live performance

Why this works:

Feature importance is relatively stable across time
You're optimizing WHAT features to use, not WHEN to use them
The rolling validation is just final confirmation

In [ ]:
%uv pip install numerapi pyarrow numerai-tools lightgbm -q

# Introduction

Lets start by loading the train and validation dataset

In [ ]:
import json
from numerapi import NumerAPI
import pandas as pd

DATA_VERSION = "v5.1"
napi = NumerAPI()

napi.download_dataset(f"{DATA_VERSION}/features.json")
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
feature_sets = feature_metadata["feature_sets"]
feature_set = feature_sets["all"]

# Download the training data - this will take a few minutes
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
train = pd.read_parquet(
    f"{DATA_VERSION}/train.parquet",
    columns=["era", "target"] + feature_set
)

# Download validation data - this will take a few minutes
napi.download_dataset(f"{DATA_VERSION}/validation.parquet")
validation = pd.read_parquet(
    f"{DATA_VERSION}/validation.parquet",
    columns=["era", "target"] + feature_set
)

# Concatenate train and validation datasets for training
train_val = pd.concat([train, validation], ignore_index=True)

#Keep the last 100th eras for test set
test_era = train_val["era"].unique()[-100:]

test = train_val[train_val["era"].isin(test_era)]
train_val = train_val[~train_val["era"].isin(test_era)]

We remove first test eras to prevent data leakage.

In [ ]:

# Eras are 1 week apart, but targets look 20 days (o 4 weeks/eras) into the future,
# so we need to "embargo" the first 4 eras following our last train era to avoid "data leakage"
last_train_era = int(train_val["era"].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(4)]]

test = test[~test["era"].isin(eras_to_embargo)]

# Training

We observe the best model is the following one:

TODO complete here

Now we are gonna train on the full train and val eras and run the model on test eras to see its actual real performance.

**NB** We should actually do rolling cross validation every week however this is costly and we don't expect it to give a major change so we are only gonna train the model once.

In [ ]:
import lightgbm as lgb
import warnings
from numerai_tools.scoring import numerai_corr
import warnings
import numpy as np


model = lgb.LGBMRegressor(
  n_estimators=2000,
  learning_rate=0.01,
  max_depth=5,
  num_leaves=2**5-1,
  colsample_bytree=0.1,
  verbose=-1,
  seed=42
  )

model.fit(train_val[feature_set], train_val["target"])
test["prediction"] = model.predict(test[feature_set])



# Compute per-era correlation
with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
    per_era_corr = validation.groupby("era").apply(
        lambda x: numerai_corr(x[["prediction"]].dropna(), x["target"].dropna()),
        include_groups=False
    )
    per_era_corr.fillna(0, inplace=True)

# Compute performance metrics
corr_mean = per_era_corr.mean()
corr_std = per_era_corr.std(ddof=0)
corr_sharpe = corr_mean / corr_std if corr_std > 0 else np.nan
corr_max_drawdown = (per_era_corr.cumsum().expanding(min_periods=1).max() - per_era_corr.cumsum()).max()
corr_hit_rate = (per_era_corr > 0).mean()


# Display performance metrics
print("=" * 60)
print("MODEL PERFORMANCE METRICS")
print("=" * 60)
print(f"Mean:           {corr_mean:>10.6f}")
print(f"Std:            {corr_std:>10.6f}")
print(f"Sharpe:         {corr_sharpe:>10.4f}")
print(f"Max Drawdown:   {corr_max_drawdown:>10.6f}")
print(f"Hit Rate:       {corr_hit_rate:>10.2%}")
print()


# Compute feature exposures
with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
    feature_exposures = validation.groupby("era").apply(
        lambda d: d.drop(columns=["target", "prediction"]).corrwith(d["prediction"]),
        include_groups=False
    )
    feature_exposures.fillna(0, inplace=True)

# Display feature exposure metrics
max_feature_exposure = feature_exposures.max(axis=1)
print("FEATURE EXPOSURE")
print("=" * 60)
print(f"Max Feature Exposure - Mean: {max_feature_exposure.mean():.4f}")
print(f"Max Feature Exposure - Std:  {max_feature_exposure.std():.4f}")
print("=" * 60)
print()


# Plot the cumulative per-era correlation
per_era_corr.cumsum().plot(
title="Cumulative Validation CORR",
kind="line",
figsize=(8, 4),
legend=False
)